In [ ]:
from collections import Counter
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import math
from scipy import optimize
#from ipywidgets import interact
#import ipywidgets as wg
import statistics as st
from scipy import special

# Funções para serem utilizadas com fits

In [ ]:
def fnt_fit_sin(x, a, b):
    return a * np.sin(b * x)

def fnt_fit_linear(x, ang, lin):
    return x*ang+lin

def fnt_fit_exp(x, a, b):
    return a * np.exp(b * x)

def fnt_fit_gauss(x, amp, cen, wid):
    #return (amp / (np.sqrt(2*np.pi) * wid)) * np.exp(-(x-cen)**2 / (2*wid**2))
    return amp * np.exp(-(x-cen)**2 / wid)

def fnt_fit_gauss_2(x, amp, cen, wid):
    #return (amp / (np.sqrt(2*np.pi) * wid)) * np.exp(-(x-cen)**2 / (2*wid**2))
    return (amp  * np.exp(-(x-cen)**2 / (2*wid**2)))


def fnt_fit_passaalta(x, cen, wid):
    return (1.0 + special.erf((x-cen) / (wid*math.sqrt(2.0)))) / 2.0

def fnt_fit_passabaixa(x, cen, wid):
    return (1 - ((1.0 + special.erf((x-cen) / (wid*math.sqrt(2.0)))) / 2.0))

def fnt_fit_passa(x, cen, wid):
    return (1)

def fnt_fit_passabanda(pe): 
    # 0-150pe passa-alta, 150-400 passa-baixa
    params_m= [36.7952354 , 63.94190579, 12.30416433]    #params para EFR de 3MeV
    params_c= [ 13.57581613, 287.66548111,  22.42501536] #params para EFR de 10MeV
    if (pe<=150):
        p = fnt_fit_passaalta(pe, params_m[1], params_m[2])
    elif (pe<=370):
        p = fnt_fit_passabaixa(pe,params_c[1],params_c[2])
    else:
        p = 0
    return (p)

def fnt_fit_decay(x, amp, cen, tau, alt):
    return (amp*np.exp(-(x-cen)/tau))+alt


## Funções de conversão

In [ ]:

def fnt_pe2MeV(x):
    return(3.77E-2*x + 0.62)

def fnt_DUQ2MeV(x):
    return(4.84E-4*x + 0.62)

def fnt_MeV2DUQ(x):
    return((x - 0.62)/4.84E-4)

### E_neutrino = raiz(E_positron^2 + 0,511^2) + 1,3
def fnt_Positron2Neutrino(x):
    return((x**2 + 0.511**2)**0.5 + 1.3)

### E_positron = raiz((E_neutrino - 1,3)^2 - 0,511^2)
def fnt_Neutrino2Positron(x):
    return(((x - 1.3)**2 - 0.511**2)**0.5)


## Funções de espectro de neutrinos por elemento combustível

In [ ]:
def fnt_spectrum_neutrino235U(energy):
    a = [4.367,-4.577,2.100, -0.5294, 0.06186, -0.002777]
    #parametro alfa da função para uranio 239 (ver ref arXiv: 1106.0687v4)
    return (np.exp(a[0]+a[1]*energy**1+a[2]*energy**2+a[3]*energy**3+a[4]*energy**4+a[5]*energy**5))

def fnt_spectrum_neutrino239Pu(energy):
    a = [4.757, -5.392, 2.563, -0.6596, 0.07820, -0.003536]
    #parametro alfa da função para uranio 239 (ver ref arXiv: 1106.0687v4)
    return (np.exp(a[0]+a[1]*energy**1+a[2]*energy**2+a[3]*energy**3+a[4]*energy**4+a[5]*energy**5))

def fnt_spectrum_neutrino241Pu(energy):
    a = [2.990, -2.882, 1.278, -0.3343, 0.03905, -0.001754]
    #parametro alfa da função para uranio 239 (ver ref arXiv: 1106.0687v4)
    return (np.exp(a[0]+a[1]*energy**1+a[2]*energy**2+a[3]*energy**3+a[4]*energy**4+a[5]*energy**5))

def fnt_spectrum_neutrino238U(energy):
    a = [0.976, -0.162, -0.0790]
    #parametro alfa da função para uranio 238 (ver ref arXiv: 0407026)
    return (np.exp(a[0]+a[1]*energy**1+a[2]*energy**2))

def fnt_cross_section(energy):
    m_eletron = 9.10938356E-31
    return(9.45E-44*(energy - 1.293)*((energy - 1.293)**2 - m_eletron**2)**0.5)
